In [ ]:
# ─────────────────────────────────────────────────────────────
# Cell 1 ▸ Imports for ANN training
# ─────────────────────────────────────────────────────────────
import joblib
from keras import callbacks, layers, models, optimizers
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


In [ ]:
print(tf.config.list_physical_devices("GPU"))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# ─────────────────────────────────────────────────────────────
#  Cell 2 ▸ Load & tidy the data
# ─────────────────────────────────────────────────────────────
csv_path = "data/processed/preprocessed_dataset.csv"
raw = pd.read_csv(csv_path)

# Harmonise column names
rename_map = {
    "Time_ms": "time",
    "Pc_bar": "chamb_pressure",
    "Tc_K": "cham_temp",
    "Pinj_bar": "injection_pres",
    "rho_kgm3": "density",
    "mu_Pas": "viscosity",
    "angle_shadow_deg": "angle_shadow",
    "len_shadow_L_D": "length_shadow",
    "angle_mie_deg": "angle_mie",
    "len_mie_L_D": "length_mie",
}
df = raw.rename(columns=rename_map)

INPUTS = ["time", "chamb_pressure", "cham_temp", "injection_pres", "density", "viscosity"]
TARGETS = ["angle_mie", "length_mie", "angle_shadow", "length_shadow"]

print("Data shape  :", df.shape)
display(df.head())

Data shape  : (726, 11)


,run,time,chamb_pressure,cham_temp,injection_pres,density,viscosity,angle_shadow,length_shadow,angle_mie,length_mie
0,ETH-01,0.000,55.03180,192.029519,98.864550,810.720228,0.001879,16.694545,13.126559,12.937325,17.571262
1,ETH-01,0.025,55.00570,192.015831,98.874062,810.718262,0.001879,16.694545,13.126559,12.937325,17.571262
2,ETH-01,0.050,55.00810,191.988228,98.907356,810.718443,0.001879,16.694545,13.126559,12.937325,17.571262
3,ETH-01,0.075,55.01635,192.081988,98.855037,810.719065,0.001879,16.694545,13.126559,12.937325,17.571262
4,ETH-01,0.100,55.01250,191.988000,98.878819,810.718775,0.001879,16.694545,20.204667,12.937325,24.506565


In [ ]:
from sklearn.model_selection import train_test_split

# ─────────────────────────────────────────────────────────────
#  Cell 3 ▸ Train-test split  (stratified by experimental run)
# ─────────────────────────────────────────────────────────────
X, y = df[INPUTS], df[TARGETS]
runs = df["run"]  # stratification label
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20, random_state=42, stratify=runs)

print(f"Train: {X_tr.shape}   Test: {X_te.shape}")


Train: (580, 6)   Test: (146, 6)


In [ ]:
# ─────────────────────────────────────────────────────────────
# Cell 4 ▸ Prepare scaled train and test data (reuse X_tr, y_tr, X_te, y_te)
# ─────────────────────────────────────────────────────────────
# Use the same train-test split as ML baselines (reuse your existing X_tr, y_tr, X_te, y_te)

# Scale input features
input_scaler = StandardScaler()
X_train_scaled = input_scaler.fit_transform(X_tr)
X_test_scaled = input_scaler.transform(X_te)

# Scale output targets
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_tr)
y_test_scaled = target_scaler.transform(y_te)

# Save scalers
joblib.dump(input_scaler, "models/ann_input_scaler.joblib")
joblib.dump(target_scaler, "models/ann_target_scaler.joblib")

print("Scaling completed. Shapes:", X_train_scaled.shape, y_train_scaled.shape)

Scaling completed. Shapes: (580, 6) (580, 4)


In [ ]:
# ─────────────────────────────────────────────────────────────
# Cell 5 ▸ Build the ANN model architecture
# ─────────────────────────────────────────────────────────────
def build_deep_ann(input_dim, output_dim):
    model = models.Sequential(
        [
            layers.InputLayer(input_shape=(input_dim,)),
            layers.Dense(256, activation="relu"),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Dense(128, activation="relu"),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(64, activation="relu"),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(output_dim, activation="linear"),
        ]
    )

    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model


model = build_deep_ann(X_train_scaled.shape[1], y_train_scaled.shape[1])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 256)            │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,996 (175.77 KB)

 Trainable params: 44,100 (172.27 KB)

 Non-trainable params: 896 (3.50 KB)

In [ ]:
# ─────────────────────────────────────────────────────────────
# Cell 6 ▸ Train the ANN model with EarlyStopping
# ─────────────────────────────────────────────────────────────
early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=30, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10)

history = model.fit(
    X_train_scaled,
    y_train_scaled,
    validation_split=0.2,
    epochs=500,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose="2",
)

Epoch 1/500
15/15 - 8s - 545ms/step - loss: 2.3214 - mae: 1.1992 - val_loss: 0.8218 - val_mae: 0.7679 - learning_rate: 1.0000e-03
Epoch 2/500
15/15 - 0s - 8ms/step - loss: 1.3951 - mae: 0.9243 - val_loss: 0.7693 - val_mae: 0.7417 - learning_rate: 1.0000e-03
Epoch 3/500
15/15 - 0s - 8ms/step - loss: 1.0165 - mae: 0.7931 - val_loss: 0.7341 - val_mae: 0.7217 - learning_rate: 1.0000e-03
Epoch 4/500
15/15 - 0s - 8ms/step - loss: 0.9531 - mae: 0.7727 - val_loss: 0.6793 - val_mae: 0.6919 - learning_rate: 1.0000e-03
Epoch 5/500
15/15 - 0s - 9ms/step - loss: 0.7438 - mae: 0.6935 - val_loss: 0.6307 - val_mae: 0.6654 - learning_rate: 1.0000e-03
Epoch 6/500
15/15 - 0s - 8ms/step - loss: 0.6664 - mae: 0.6440 - val_loss: 0.5954 - val_mae: 0.6485 - learning_rate: 1.0000e-03
Epoch 7/500
15/15 - 0s - 8ms/step - loss: 0.6121 - mae: 0.6167 - val_loss: 0.5577 - val_mae: 0.6259 - learning_rate: 1.0000e-03
Epoch 8/500
15/15 - 0s - 8ms/step - loss: 0.5931 - mae: 0.5990 - val_loss: 0.5286 - val_mae: 0.6064 - 

In [ ]:
# ─────────────────────────────────────────────────────────────
# Cell 7 ▸ Evaluate the ANN model on the test set
# ─────────────────────────────────────────────────────────────

# Inverse transform predictions & evaluate on original scale
y_pred_scaled = model.predict(X_test_scaled)
y_pred = target_scaler.inverse_transform(y_pred_scaled)

r2_scores, mae_scores, mse_scores = [], [], []
for i, col in enumerate(y_te.columns):
    r2 = r2_score(y_te[col], y_pred[:, i])
    mae = mean_absolute_error(y_te[col], y_pred[:, i])
    mse = mean_squared_error(y_te[col], y_pred[:, i])
    r2_scores.append(r2)
    mae_scores.append(mae)
    mse_scores.append(mse)
    print(f"{col:>15}: R²={r2:.4f}, MAE={mae:.4f}, MSE={mse:.4f}")

print(f"\nOverall mean R²: {np.mean(r2_scores):.4f}")
print(f"Overall mean MAE: {np.mean(mae_scores):.4f}")
print(f"Overall mean MSE: {np.mean(mse_scores):.4f}")
# Save model & predictions (original scale)
model.save("models/ANN_improved_regressor.h5")
pd.DataFrame(y_pred, columns=y_te.columns).to_csv(
    "outputs/ANN_improved_predictions.csv", index=False
)
y_te.to_csv("outputs/ANN_improved_actuals.csv", index=False)
# Save ANN metrics for each target to CSV
ann_metrics = pd.DataFrame(
    {
        "Model": ["ANN"] * len(y_te.columns),
        "target": list(y_te.columns),
        "r2": r2_scores,
        "mae": mae_scores,
        "mse": mse_scores,
    }
)
ann_metrics.to_csv("outputs/ANN_improved_metrics.csv", index=False)
print("Saved improved model and predictions.")

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step


      angle_mie: R²=0.9910, MAE=0.0954, MSE=0.0140
     length_mie: R²=0.9924, MAE=3.3012, MSE=20.5885
   angle_shadow: R²=0.9849, MAE=0.1192, MSE=0.0216
  length_shadow: R²=0.9932, MAE=3.1945, MSE=20.0414

Overall mean R²: 0.9904
Overall mean MAE: 1.6776
Overall mean MSE: 10.1664

Saved improved model and predictions.
